In [2]:
import time

# Random HUBO generator

In [3]:

import numpy as np

def generate_random_hubo(num_variables, max_order, num_terms, seed=None):
    # Args:
    #    num_variables (int): The total number of binary variables in the HUBO problem.
    #    max_order (int):  The maximum order (degree) of terms in the HUBO problem. 
    #    num_terms (int): The number of terms to generate in the HUBO problem.
    #    seed (int, optional): Random seed for reproducibility. Defaults to None.

    rng = np.random.default_rng(seed)
    hubo = {} # Dict to store the HUBO problem

    for _ in range(num_terms):
        # Randomly choose the order of the term
        order = rng.integers(1, max_order + 1)

        # Randomly select variable indices for the term
        variables = tuple(sorted(rng.choice(num_variables, size=order, replace=False)))

        # Randomly generate a coefficient for the term
        coefficient = rng.uniform(-10, 10)

        # Add the term to the HUBO problem
        hubo[variables] = coefficient

    # Returns:
    #    dict: The generated HUBO problem as a dictionary.
    #          Keys are tuples of variable indices, values are coefficients.

    return hubo


# Example usage
num_variables = 3    # Number of binary variables
max_order = 3        # Maximum order of terms (here cubic terms) --> If set to 2 we get a QUBO problem!
num_terms = 6       # Number of terms in the HUBO
seed = 1            # Random seed for reproducibility of terms and coefficients

rand_hubo_problem = generate_random_hubo(num_variables, max_order, num_terms, seed)

# Print the HUBO problem
print("Generated HUBO Problem:")
for key, value in rand_hubo_problem.items():
    print(f"Term: {key}, Coefficient: {value:.2f}")




Generated HUBO Problem:
Term: (1, 2), Coefficient: -3.41
Term: (0, 1, 2), Coefficient: 6.55
Term: (0, 1), Coefficient: -9.45
Term: (0, 2), Coefficient: -0.93
Term: (2,), Coefficient: -1.94


In [4]:
# Print the HUBO problem in its dictionary format
rand_hubo_problem

{(1, 2): -3.4053656700181563,
 (0, 1, 2): 6.554051876408835,
 (0, 1): -9.448817735138633,
 (0, 2): -0.9300422103869703,
 (2,): -1.9377402710574145}

In [5]:
import numpy as np

def map_hubo_to_qubo(hubo, num_variables):
    """
    Map a HUBO problem with higher-order terms to a QUBO problem.
    
    Args:
    - hubo (dict): HUBO problem with higher-order terms as key-value pairs.
    - num_variables (int): Number of binary variables in the HUBO problem.
    
    Returns:
    - qubo_matrix (numpy.ndarray): The resulting QUBO matrix.
    """
    # Initialize QUBO matrix
    qubo_matrix = np.zeros((num_variables, num_variables))
    
    aux_var_index = num_variables  # Start index for auxiliary variables
    
    for variables, coeff in hubo.items():
        order = len(variables)
        
        if order == 1:  # Linear terms
            i = variables[0]
            qubo_matrix[i, i] += coeff
            
        elif order == 2:  # Quadratic terms
            i, j = variables
            qubo_matrix[i, j] += coeff
            qubo_matrix[j, i] += coeff  # Symmetric matrix
            
        elif order == 3:  # Cubic terms
            i, j, k = variables
            z_ijk = aux_var_index
            aux_var_index += 1

            # Resize the QUBO matrix to accommodate the auxiliary variable
            if aux_var_index >= qubo_matrix.shape[0]:
                qubo_matrix = np.resize(qubo_matrix, (aux_var_index + 1, aux_var_index + 1))
            
            # Add penalty terms for cubic interaction
            penalty_coeff =  coeff  # Penalty coefficient
            qubo_matrix = add_penalty_terms(qubo_matrix, z_ijk, [i, j, k], penalty_coeff, cubic=True)
            
            # Update QUBO matrix for cubic term
            qubo_matrix[i, j] += coeff * z_ijk
            qubo_matrix[j, k] += coeff * z_ijk
            qubo_matrix[i, k] += coeff * z_ijk
            
        # Additional cases can be added for quartic terms or higher-order terms.
    
    return qubo_matrix

def add_penalty_terms(Q, aux_var, vars, penalty_coeff, cubic=True):
    """
    Add penalty terms to the QUBO matrix for enforcing higher-order terms (cubic or quartic).
    
    Args:
    - Q: The QUBO matrix.
    - aux_var: The index of the auxiliary variable.
    - vars: The indices of the variables involved in the interaction.
    - penalty_coeff: The penalty coefficient.
    - cubic: Whether the penalty is for cubic (True) or quartic (False).
    
    Returns:
    - Updated QUBO matrix.
    """
    if cubic:
        # For cubic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff
    else:
        # For quartic terms, introduce quadratic penalties
        Q[aux_var, aux_var] = penalty_coeff
        for v in vars:
            Q[aux_var, v] = -penalty_coeff
            Q[v, aux_var] = -penalty_coeff

    return Q

def matr_to_dict(input_matrix):
    upper_triangle_dict = {(i, j): qubo_matrix[i][j] 
                            for i in range(len(qubo_matrix)) 
                            for j in range(i, len(qubo_matrix[i]))}  # Only for j >= i as we should have a symmetrical matrix
    return upper_triangle_dict

# Number of binary variables
num_variables = 3

# Convert HUBO problem to QUBO matrix
qubo_matrix = map_hubo_to_qubo(rand_hubo_problem, num_variables)
qubo_dict = matr_to_dict(qubo_matrix)

# Print the resulting QUBO Matrix
print("Converted QUBO Matrix:")
print(qubo_matrix)
# Print the resulting QUBO dictionary
print("Converted QUBO dictionary:")
print(qubo_dict)


Converted QUBO Matrix:
[[  0.          10.21333789  18.73211342  -6.55405188   0.        ]
 [-12.85418341   0.          16.25678996  -6.55405188   0.        ]
 [ -0.93004221   0.          -1.93774027  -6.55405188  -3.40536567]
 [ -6.55405188  -6.55405188  -6.55405188   6.55405188   0.        ]
 [  0.           0.           0.          -3.40536567   0.        ]]
Converted QUBO dictionary:
{(0, 0): 0.0, (0, 1): 10.213337894087871, (0, 2): 18.732113418839532, (0, 3): -6.554051876408835, (0, 4): 0.0, (1, 1): 0.0, (1, 2): 16.256789959208348, (1, 3): -6.554051876408835, (1, 4): 0.0, (2, 2): -1.9377402710574145, (2, 3): -6.554051876408835, (2, 4): -3.4053656700181563, (3, 3): 6.554051876408835, (3, 4): 0.0, (4, 4): 0.0}


In [6]:
import numpy as np
import dimod

# Define the QUBO matrix
# Example QUBO matrix for the problem: minimize x0^2 + x0*x1 - 2*x1^2
Q = {
    ('x0', 'x0'): 1,    # x0^2 term
    ('x0', 'x1'): 1,    # x0*x1 term
    ('x1', 'x1'): -2    # x1^2 term
}

# Use dimod to define a QUBO model
qubo = dimod.BinaryQuadraticModel.from_qubo(qubo_dict)

# Solve the QUBO classically using a brute force solver
class_sampler = dimod.ExactSolver()
class_solution = class_sampler.sample(qubo)

# Extract the results
print("Solutions:")
for sample, energy in solution.data(['sample', 'energy']):
    print(f"Sample: {class_sampler}, Energy: {energy}")

# Find the optimal solution
optimal_class_sampler = solution.first.sample
optimal_class_energy = solution.first.energy

Solutions:


NameError: name 'solution' is not defined

In [7]:
import itertools

def evaluate_qubo(qubo, assignment):
    
    #Evaluate the QUBO objective function for a given binary assignment.

    #Args:
    #    qubo (dict): The QUBO problem as a dictionary.
    #    assignment (dict): A dictionary mapping variable names to binary values (0 or 1).

    #Returns:
    #    float: The value of the objective function.

    # Evaluate the objective function by multiplying the coefficients with the assigned values and summing them up.

    # Note: The QUBO is in the form of a dictionary where keys are tuples of variable names (e.g., ('x0', 'x1')) and values are coefficients.
    # This function assumes that the QUBO is symmetric, i.e., qubo[(v1, v2)] == qubo[(v2, v1)].

    # Example:
    objective_value = 0
    for (vars_, coeff) in qubo.items():
        product = coeff
        for var in vars_:
            product *= assignment[var]
        objective_value += product
    return objective_value

start_brute = time.time()
def brute_force_solve_qubo(qubo):
    
    #Solve the QUBO problem using a brute-force approach.

    #Args:
    #    qubo (dict): The QUBO problem as a dictionary.

    #Returns:
    #    dict: The optimal binary assignment.
    #    float: The minimum value of the objective function.

    # Extract all unique variables from the QUBO dictionary keys.
    variables = set()
    for key in qubo.keys():
        variables.update(key)

    # Generate all possible binary assignments
    num_variables = len(variables)
    variable_list = list(variables)
    all_assignments = itertools.product([0, 1], repeat=num_variables)

    # Evaluate all assignments and find the minimum
    min_value = float('inf')    # value of the minimum
    optimal_assignment = []   # the optimal binary assignment of variables

    for assignment_tuple in all_assignments:
        assignment = dict(zip(variable_list, assignment_tuple))
        value = evaluate_qubo(qubo, assignment)
        if value < min_value:
            min_value = value
            optimal_assignment.append(assignment)

    return optimal_assignment, min_value

# Solve the QUBO problem
optimal_assignment_hubo, min_value_hubo = brute_force_solve_qubo(rand_hubo_problem)
optimal_assignment_qubo, min_value_qubo = brute_force_solve_qubo(qubo_dict)


end_brute = time.time()
# Print the results
print("Optimal Assignment(s):")
for el in optimal_assignment_hubo:
    print(el)

print("Minimum Objective Value:", min_value_hubo)
print("Calculated in:", end_brute - start_brute, "seconds.")


Optimal Assignment(s):
{0: 0, 1: 0, 2: 0}
{0: 0, 1: 0, 2: 1}
{0: 0, 1: 1, 2: 1}
{0: 1, 1: 1, 2: 0}
Minimum Objective Value: -9.448817735138633
Calculated in: 0.0 seconds.


In [8]:
# Print the results
print("Optimal Assignment(s):")
for el in optimal_assignment_qubo:
    print(el)

print("Minimum Objective Value:", min_value_qubo)
print("Calculated in:", end_brute - start_brute, "seconds.")

Optimal Assignment(s):
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
{0: 0, 1: 0, 2: 1, 3: 0, 4: 0}
{0: 0, 1: 0, 2: 1, 3: 0, 4: 1}
Minimum Objective Value: -5.343105941075571
Calculated in: 0.0 seconds.
